In [1]:
import os

In [2]:
%pwd

'/config/workspace/End-to-end-ML-project/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/config/workspace/End-to-end-ML-project'

In [5]:
# update entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    unzip_data_dir: str
    STATUS_FILE: Path
    all_schema: dict

In [6]:
# update the configuration manager in src config

from red_wine_quality_project.constants import *
from red_wine_quality_project.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir = config.root_dir,
            STATUS_FILE = config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema = schema,
        )

        return data_validation_config

In [7]:
# update the components

import os
from red_wine_quality_project import logger
import pandas as pd


class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e

# you can also validate the datatype by creating new function validate_datatype(self)-> bool

In [8]:
# update the pipeline

try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2024-01-07 12:48:51,090: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-01-07 12:48:51,092: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-07 12:48:51,096: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-01-07 12:48:51,097: INFO: common: created directory at: artifacts]
[2024-01-07 12:48:51,098: INFO: common: created directory at: artifacts/data_validation]
